In [1]:
import os
import json
import time
import wbdata
import pymongo
import requests
import datetime
import psycopg2
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from itertools import product
from scipy.stats import gmean
from newspaper import Config, Article, fulltext
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# Define a dictionary that maps the World Bank indicator code for GDP to a human-readable name
sector_dict = {"NY.GDP.MKTP.CD": "GDP (current US$)"}

# Get a list of country IDs for all high-income countries
countries = [i['id'] for i in wbdata.get_country(incomelevel="HIC")]

# Define the date for which we want to retrieve the GDP data (January 1, 2021)
data_date = (datetime.datetime(2021, 1, 1))

# Use the World Bank API to retrieve the GDP data for all sectors, countries, and years
gdp = wbdata.get_dataframe(sector_dict, country=countries, data_date=data_date)

# Print the the resulting DataFrame to the console
gdp.to_csv('gdp.csv', index=True)
gdp

,GDP (current US$)
country,
Aruba,3.126019e+09
Andorra,3.330282e+09
United Arab Emirates,4.150216e+11
Antigua and Barbuda,1.471126e+09
Australia,1.552667e+12
...,...
Trinidad and Tobago,2.446020e+10
Uruguay,5.931948e+10
United States,2.331508e+13


In [3]:
# Sort the GDP DataFrame by the 'GDP (current US$)' column in descending order
gdp_sorted = gdp.sort_values(by='GDP (current US$)', ascending=False)

# Keep only the top 20 countries by GDP
top_25_gdp = gdp_sorted.head(25)

# Print the resulting DataFrame to the console
top_25_gdp.to_csv('top_25_gdp.csv', index=True)
top_25_gdp

,GDP (current US$)
country,
United States,2.331508e+13
Japan,4.940878e+12
Germany,4.259935e+12
United Kingdom,3.131378e+12
France,2.957880e+12
Italy,2.107703e+12
Canada,1.988336e+12
"Korea, Rep.",1.810956e+12
Australia,1.552667e+12


In [4]:
# Define a dictionary that maps the World Bank indicator code for GDP growth to a human-readable name
sector_dict = {"NY.GDP.MKTP.KD.ZG": "GDP growth (annual %)"}

# Get a list of country IDs for all high-income countries
countries = [i['id'] for i in wbdata.get_country(incomelevel="HIC")]

# Define the date range for which we want to retrieve the GDP growth data (2016-2021)
data_date = (datetime.datetime(2016, 1, 1), datetime.datetime(2021, 1, 1))

# Use the World Bank API to retrieve the GDP growth data for all sectors, countries, and years
gdp_growth = wbdata.get_dataframe(sector_dict, country=countries, data_date=data_date)

# Print the resulting DataFrame to the console
gdp_growth

GDP growth (annual %)
country               date                       
Aruba                 2021              17.172509
                      2020             -18.589100
                      2019               0.635029
                      2018               5.257856
                      2017               5.482371
...                                           ...
Virgin Islands (U.S.) 2020              -2.147007
                      2019               2.777778
                      2018               1.904762
                      2017              -0.735294
                      2016               1.600854

[480 rows x 1 columns]

In [5]:
# Unstack the 'GDP growth (annual %)' column of the gdp_growth DataFrame to create a multi-indexed DataFrame
gdp_growth = gdp_growth['GDP growth (annual %)'].unstack()

# Compute the mean GDP growth rate across all years for each country
gdp_growth = gdp_growth.mean(axis=1)

# Print the resulting Series object to the console
gdp_growth.to_csv('gdp_growth.csv', index=True)
gdp_growth

country
Andorra                  0.904257
Antigua and Barbuda      0.910856
Aruba                    2.012563
Australia                2.042083
Austria                  1.048944
                           ...   
United Arab Emirates     1.279678
United Kingdom           0.735360
United States            2.054483
Uruguay                  0.398573
Virgin Islands (U.S.)    0.680219
Length: 80, dtype: float64

In [6]:
# Sort the gdp_growth Series in descending order
gdp_growth_sorted = gdp_growth.sort_values(ascending=False)

# Keep only the top 25 countries with the highest GDP growth rates
top_25_gdp_growth = gdp_growth_sorted.head(25)

# Convert the resulting Series to a DataFrame
top_25_gdp_growth = top_25_gdp_growth.to_frame()

# Rename the column to "GDP growth (annual %)"
top_25_gdp_growth = top_25_gdp_growth.rename(columns={0: 'GDP growth (annual %)'})

# Print the resulting DataFrame to the console
top_25_gdp_growth.to_csv('top_25_gdp_growth.csv', index=True)
top_25_gdp_growth

,GDP growth (annual %)
country,
Ireland,7.458877
Northern Mariana Islands,5.473814
Malta,4.727709
Cyprus,4.291799
Estonia,3.988996
Israel,3.963240
Poland,3.886061
Romania,3.726475
Monaco,3.566825


In [7]:
# Merge top_25_gdp and top_25_gdp_growth DataFrames on 'country' column using an inner join
top_countries = pd.merge(top_25_gdp, top_25_gdp_growth, on='country', how='inner')

# Print the resulting DataFrame to the console
top_countries.to_csv('top_countries.csv', index=True)
top_countries

,GDP (current US$),GDP growth (annual %)
country,,
United States,2.331508e+13,2.054483
"Korea, Rep.",1.810956e+12,2.448968
Poland,6.794448e+11,3.886061
Ireland,5.041826e+11,7.458877
Israel,4.885265e+11,3.963240
Denmark,3.983033e+11,2.068636
Singapore,3.969869e+11,2.741828
Chile,3.170585e+11,2.260217


In [8]:
# Define a dictionary that maps the World Bank indicator code for Ease of Doing Business scores to a human-readable name
sector_dict = {"IC.BUS.DFRN.XQ": "Ease of doing business score (0 = lowest performance to 100 = best performance)"}

# Get a list of country IDs for all high-income countries
countries = [i['id'] for i in wbdata.get_country(incomelevel="HIC")]

# Define the date for which we want to retrieve the Ease of Doing Business scores (January 1st, 2019)
data_date = (datetime.datetime(2019, 1, 1))

# Use the World Bank API to retrieve the Ease of Doing Business scores for all sectors, countries, and years
ease_business = wbdata.get_dataframe(sector_dict, country=countries, data_date=data_date)

# Filter the resulting DataFrame to keep only the countries with scores higher than 77
ease_business = ease_business[ease_business["Ease of doing business score (0 = lowest performance to 100 = best performance)"] > 83]

# Print the resulting DataFrame to the console
ease_business.to_csv('ease_business.csv', index=True)
ease_business

,Ease of doing business score (0 = lowest performance to 100 = best performance)
country,
Denmark,85.28856
United Kingdom,83.54968
"Hong Kong SAR, China",85.31540
"Korea, Rep.",84.00083
New Zealand,86.76465
Singapore,86.19810
United States,83.99668


In [9]:
# Merge top_countires and ease_business DataFrames on 'country' column using an inner join
top_countries = pd.merge(top_countries, ease_business, on='country', how='inner')

# Print the resulting DataFrame to the console
top_countries.to_csv('top_countries_3.csv', index=True)
top_countries

,GDP (current US$),GDP growth (annual %),Ease of doing business score (0 = lowest performance to 100 = best performance)
country,,,
United States,2.331508e+13,2.054483,83.99668
"Korea, Rep.",1.810956e+12,2.448968,84.00083
Denmark,3.983033e+11,2.068636,85.28856
Singapore,3.969869e+11,2.741828,86.19810


In [10]:
# Extract the country names from the index of the top_countires DataFrame and store them in a list
countries_analysis = top_countries.index.tolist()

# Print the resulting list to the console
countries_analysis

['United States', 'Korea, Rep.', 'Denmark', 'Singapore']

In [11]:
# API endpoint for the list of countries
url = "https://api.worldbank.org/v2/country/"

# Query parameters for the API request
params = {
    "format": "json", # response format
    "per_page": 500, # number of countries per page
    "page": 1, # start from page 1
}

# Send API request to get the list of countries
response = requests.get(url, params=params)

# Extract the list of country codes from the response
countries = [c["id"] for c in response.json()[1]]

# Print the list of country codes
print(countries)

['ABW', 'AFE', 'AFG', 'AFR', 'AFW', 'AGO', 'ALB', 'AND', 'ARB', 'ARE', 'ARG', 'ARM', 'ASM', 'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEA', 'BEC', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHI', 'BHR', 'BHS', 'BIH', 'BLA', 'BLR', 'BLZ', 'BMN', 'BMU', 'BOL', 'BRA', 'BRB', 'BRN', 'BSS', 'BTN', 'BWA', 'CAA', 'CAF', 'CAN', 'CEA', 'CEB', 'CEU', 'CHE', 'CHI', 'CHL', 'CHN', 'CIV', 'CLA', 'CME', 'CMR', 'COD', 'COG', 'COL', 'COM', 'CPV', 'CRI', 'CSA', 'CSS', 'CUB', 'CUW', 'CYM', 'CYP', 'CZE', 'DEA', 'DEC', 'DEU', 'DFS', 'DJI', 'DLA', 'DMA', 'DMN', 'DNF', 'DNK', 'DNS', 'DOM', 'DSA', 'DSF', 'DSS', 'DZA', 'EAP', 'EAR', 'EAS', 'ECA', 'ECS', 'ECU', 'EGY', 'EMU', 'ERI', 'ESP', 'EST', 'ETH', 'EUU', 'FCS', 'FIN', 'FJI', 'FRA', 'FRO', 'FSM', 'FXS', 'GAB', 'GBR', 'GEO', 'GHA', 'GIB', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC', 'GRD', 'GRL', 'GTM', 'GUM', 'GUY', 'HIC', 'HKG', 'HND', 'HPC', 'HRV', 'HTI', 'HUN', 'IBB', 'IBD', 'IBT', 'IDA', 'IDB', 'IDN', 'IDX', 'IMN', 'IND', 'INX', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA', 'JAM'

In [12]:
# API endpoint for the list of countries
url = "https://api.worldbank.org/v2/country/"

# Query parameters for the API request
params = {
    "format": "json", # response format
    "per_page": 500, # number of countries per page
    "page": 1, # start from page 1
}

# Send API request to get the list of countries
response = requests.get(url, params=params)

# Extract the list of country names from the response
countries_name = [c["name"] for c in response.json()[1]]

# Print the list of country names
print(countries_name)

['Aruba', 'Africa Eastern and Southern', 'Afghanistan', 'Africa', 'Africa Western and Central', 'Angola', 'Albania', 'Andorra', 'Arab World', 'United Arab Emirates', 'Argentina', 'Armenia', 'American Samoa', 'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan', 'Burundi', 'East Asia & Pacific (IBRD-only countries)', 'Europe & Central Asia (IBRD-only countries)', 'Belgium', 'Benin', 'Burkina Faso', 'Bangladesh', 'Bulgaria', 'IBRD countries classified as high income', 'Bahrain', 'Bahamas, The', 'Bosnia and Herzegovina', 'Latin America & the Caribbean (IBRD-only countries)', 'Belarus', 'Belize', 'Middle East & North Africa (IBRD-only countries)', 'Bermuda', 'Bolivia', 'Brazil', 'Barbados', 'Brunei Darussalam', 'Sub-Saharan Africa (IBRD-only countries)', 'Bhutan', 'Botswana', 'Sub-Saharan Africa (IFC classification)', 'Central African Republic', 'Canada', 'East Asia and the Pacific (IFC classification)', 'Central Europe and the Baltics', 'Europe and Central Asia (IFC classification)

In [13]:
# List of codes of sub-sector or sub-category
sector_codes = [
    "NV.AGR.TOTL.ZS",
    "NV.IND.FORE.ZS",
    "NV.AGR.FISH.ZS",
    "NV.IND.FBTW.ZS",
    "NV.IND.TXTC.ZS",
    "NV.IND.CHEM.ZS",
    "NV.IND.PHAR.ZS",
    "NV.IND.RUBB.ZS",
    "NV.IND.NMTC.ZS",
    "NV.IND.METL.ZS",
    "NV.IND.MACH.ZS",
    "NV.IND.TRAN.ZS",
    "NV.IND.OTHR.ZS",
    "NV.MIN.COAL.ZS",
    "NV.MIN.OILG.ZS",
    "NV.MIN.METL.ZS",
    "IS.IC.BCON.ZS",
    "IS.IC.ENGR.ZS",
    "NV.IND.ENER.ZS",
    "NV.IND.ELEC.ZS",
    "NV.SRV.WHOL.ZS",
    "NV.SRV.WHOLE.ZS",
    "NV.SRV.RETL.ZS",
    "NV.SRV.TRAN.ZS",
    "IS.AIR.GOOD.MT.K1",
    "IS.SHIP.TOTL.DW",
    "IS.RRS.GOOD.MT.K1",
    "IS.ROD.GOOD.MT.K1",
    "IS.CPU.PUBS.P3",
    "NV.SRV.TETC.ZS",
    "NV.SRV.PUBL.ZS",
    "NV.SRV.CREA.ZS",
    "NV.SRV.TELC.ZS",
    "NV.SRV.SOFT.ZS",
    "NV.SRV.FINN.ZS",
    "NV.FIN.TOTL.ZS",
    "NV.SRV.OTHR.ZS",
    "NV.SRV.REAL.ZS",
    "NV.SRV.PRSC.ZS",
    "NV.SRV.LEGL.ZS",
    "NV.SRV.MANC.ZS",
    "NV.SRV.ARCH.ZS",
    "NV.SRV.ADVT.ZS",
    "NV.SRV.OTHC.ZS",
    "NV.SRV.PADM.ZS",
    "NV.SRV.FPUB.ZS",
    "NV.SRV.EDUC.ZS",
    "NV.SRV.OTHO.ZS",
    "NV.EDU.TOTL.ZS",
    "NV.SRV.HTSP.ZS",
    "NV.HEA.TOTL.ZS",
    "NV.SRV.RSCL.ZS",
    "NV.SRV.SECO.ZS",
    "NV.SRV.OTCM.ZS",
    "NV.SRV.MEMB.ZS",
    "NV.SRV.REPR.ZS",
    "NV.SRV.OTHR.ZS",
    "NV.SRV.DOMS.ZS"
]

In [14]:
# List of names of sub-sector or sub-category
sector_names = [
    "Agricultural crops and livestock",    
    "Forestry and logging",
    "Fishing and aquaculture",
    "Food, beverages, and tobacco",
    "Textiles and clothing",
    "Chemicals and chemical products",
    "Pharmaceuticals",
    "Rubber and plastics products",
    "Non-metallic mineral products",
    "Basic metals and fabricated metal products",
    "Machinery and equipment",
    "Transport equipment",
    "Other manufacturing",
    "Coal and lignite mining",
    "Oil and gas extraction",
    "Metal ores and non-metallic mining and quarrying",
    "Building construction",
    "Heavy and civil engineering construction",
    "Electricity, gas and water supply",
    "Energy and water utilities",
    "Wholesale and retail trade and repair of motor vehicles and motorcycles",
    "Wholesale trade, except of motor vehicles and motorcycles",
    "Retail trade, except of motor vehicles and motorcycles",
    "Transport and storage",
    "Air transport",
    "Water transport",
    "Railways",
    "Road transport",
    "Postal and courier activities",
    "Information and communication",
    "Publishing activities",
    "Motion picture, video, and television program production, sound recording, and music publishing activities",
    "Telecommunications",
    "Computer programming, consultancy, and related activities",
    "Financial and insurance activities",
    "Monetary intermediation",
    "Activities auxiliary to financial services and insurance activities",
    "Real estate activities",
    "Professional, scientific, and technical activities",
    "Legal and accounting and management activities",
    "Management consultancy activities",
    "Architectural and engineering activities, and related technical consultancy",
    "Advertising and market research", 
    "Other professional, scientific, and technical activities",
    "Public administration and defense; compulsory social security",
    "Public administration and defense; financial and fiscal affairs",
    "Public administration and defense; education, health, and social work", 
    "Public administration and defense; other social services",
    "Education",
    "Health and social work",
    "Human health activities",
    "Residential care activities",
    "Social work activities without accommodation",
    "Other community, social, and personal services",
    "Membership organizations",
    "Repair of computers and personal and household goods",
    "Other personal service activities",
    "Domestic and household services"
]

In [15]:
# Combine two lists of equal length (sector_codes and sector_names) into a list of tuples using the zip() function
sector_tuples = zip(sector_codes, sector_names)

# Use the resulting list of tuples to create a dictionary where the sector codes are the keys and the sector names are the values
sector_dict = dict(sector_tuples)

# Print the resulting dictionary to the console
sector_dict

{'NV.AGR.TOTL.ZS': 'Agricultural crops and livestock',
 'NV.IND.FORE.ZS': 'Forestry and logging',
 'NV.AGR.FISH.ZS': 'Fishing and aquaculture',
 'NV.IND.FBTW.ZS': 'Food, beverages, and tobacco',
 'NV.IND.TXTC.ZS': 'Textiles and clothing',
 'NV.IND.CHEM.ZS': 'Chemicals and chemical products',
 'NV.IND.PHAR.ZS': 'Pharmaceuticals',
 'NV.IND.RUBB.ZS': 'Rubber and plastics products',
 'NV.IND.NMTC.ZS': 'Non-metallic mineral products',
 'NV.IND.METL.ZS': 'Basic metals and fabricated metal products',
 'NV.IND.MACH.ZS': 'Machinery and equipment',
 'NV.IND.TRAN.ZS': 'Transport equipment',
 'NV.IND.OTHR.ZS': 'Other manufacturing',
 'NV.MIN.COAL.ZS': 'Coal and lignite mining',
 'NV.MIN.OILG.ZS': 'Oil and gas extraction',
 'NV.MIN.METL.ZS': 'Metal ores and non-metallic mining and quarrying',
 'IS.IC.BCON.ZS': 'Building construction',
 'IS.IC.ENGR.ZS': 'Heavy and civil engineering construction',
 'NV.IND.ENER.ZS': 'Electricity, gas and water supply',
 'NV.IND.ELEC.ZS': 'Energy and water utilities',

In [16]:
# PESTEL Analysis Categories
analysis = [
    "Political News Article",    
    "Economical News Article",
    "Social News Article",
    "Technological News Article",
    "Environmental News Article",
    "Legal News Article"
]

In [17]:
# Use itertools.product() to create all possible combinations of countries, sectors, and categories to study PESTEL factors
combinations = list(product(countries_analysis, sector_names, analysis))

# Convert the resulting list of tuples to a pandas DataFrame called 'news'
news = pd.DataFrame(combinations, columns=['Country', 'Sector', 'Analysis'])

# Add a new column to the DataFrame called 'search', which combines the values in each row using the apply() method and a lambda function
news['search'] = news.apply(lambda x: ' '.join(x), axis=1)

# Print the resulting DataFrame to the console
news

,Country,Sector,Analysis,search
0,United States,Agricultural crops and livestock,Political News Article,United States Agricultural crops and livestock...
1,United States,Agricultural crops and livestock,Economical News Article,United States Agricultural crops and livestock...
2,United States,Agricultural crops and livestock,Social News Article,United States Agricultural crops and livestock...
3,United States,Agricultural crops and livestock,Technological News Article,United States Agricultural crops and livestock...
4,United States,Agricultural crops and livestock,Environmental News Article,United States Agricultural crops and livestock...
...,...,...,...,...
1387,Singapore,Domestic and household services,Economical News Article,Singapore Domestic and household services Econ...
1388,Singapore,Domestic and household services,Social News Article,Singapore Domestic and household services Soci...
1389,Singapore,Domestic and household services,Technological News Article,Singapore Domestic and household services Tech...
1390,Singapore,Domestic and household services,Environmental News Article,Singapore Domestic and household services Envi...


In [18]:
# Extract the values from the 'search' column of the news DataFrame and store them in a list
search_terms = news['search'].tolist()

# Print the resulting list to the console
search_terms

['United States Agricultural crops and livestock Political News Article',
 'United States Agricultural crops and livestock Economical News Article',
 'United States Agricultural crops and livestock Social News Article',
 'United States Agricultural crops and livestock Technological News Article',
 'United States Agricultural crops and livestock Environmental News Article',
 'United States Agricultural crops and livestock Legal News Article',
 'United States Forestry and logging Political News Article',
 'United States Forestry and logging Economical News Article',
 'United States Forestry and logging Social News Article',
 'United States Forestry and logging Technological News Article',
 'United States Forestry and logging Environmental News Article',
 'United States Forestry and logging Legal News Article',
 'United States Fishing and aquaculture Political News Article',
 'United States Fishing and aquaculture Economical News Article',
 'United States Fishing and aquaculture Social Ne

In [ ]:
# Set the subscription key and endpoint for the Bing Search API
subscription_key = ''
endpoint = 'https://api.bing.microsoft.com/' + "v7.0/search"

# Set the market (mkt) parameter for the Bing Search API request
mkt = 'en-US'

# Set the request headers to include the subscription key
headers = {'Ocp-Apim-Subscription-Key': subscription_key}

# Initialize an empty list to store the results
results = []

# Loop over each search term and send a request to the Bing Search API
for query in search_terms:
    # Construct the request parameters for the Bing Search API
    params = {'q': query, 'mkt': mkt}

    try:
        # Send a request to the Bing Search API and parse the JSON response
        response = requests.get(endpoint, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()

        # Loop over each result in the JSON response and append it to the results list
        for result in data['webPages']['value']:
            results.append({
                'query': query,
                'title': result['name'],
                'url': result['url']
            })
    except Exception as ex:
        # Print an error message if the request fails
        print(f"Error searching for {query}: {ex}")

# Create a pandas DataFrame from the results list
df_news = pd.DataFrame(results)

# Print the first five rows of the resulting DataFrame to the console
df_news

# The result of this has already been exported, to avoid double use of the API

In [19]:
# Import Trained Database from Twitter http://help.sentiment140.com/for-students/

# Define a list of possible encodings to try
encodings = ['utf-8', 'ISO-8859-1', 'Windows-1252', 'ASCII']

# Loop over each encoding and attempt to read the CSV file
for encoding in encodings:
    try:
        # Attempt to read the CSV file using the current encoding
        data = pd.read_csv('training_twitter.csv', encoding=encoding)

        # Print a success message to the console and break out of the loop if the file is successfully read
        print(f"File read successfully with encoding {encoding}")
        break
    except Exception as ex:
        # Print an error message to the console if the file cannot be read with the current encoding
        print(f"Error reading file with encoding {encoding}: {ex}")

# Print the resulting DataFrame to the console
data

Error reading file with encoding utf-8: 'utf-8' codec can't decode bytes in position 232719-232720: invalid continuation byte
File read successfully with encoding ISO-8859-1


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [20]:
# Get a list of the current column names in the DataFrame
columns = list(data.columns)

# Modify the names of specific columns
columns[0] = 'sentiment'
columns[1] = 'NUMBER'
columns[2] = 'DATE'
columns[3] = 'NO_QUERY'
columns[4] = 'USERNAME'
columns[5] = 'title'

# Set the column names in the DataFrame to the modified list
data.columns = columns

# Print the resulting DataFrame to the console
data

,sentiment,NUMBER,DATE,NO_QUERY,USERNAME,title
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [21]:
# Define a dictionary that maps specific sentiment values to standardized values, -1 negative and 1 positive
sentiment_map = {0: -1, 2: 1, 4: 1}

# Apply the mapping dictionary to the 'sentiment' column of the DataFrame using the map() method
data['sentiment'] = data['sentiment'].map(sentiment_map)

# Print the resulting DataFrame to the console (optional)
data

,sentiment,NUMBER,DATE,NO_QUERY,USERNAME,title
0,-1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,-1,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,-1,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,-1,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,-1,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [22]:
# Using the df alredy saved from the BING API

# Load the data into a pandas DataFrame
df_news_excel = pd.read_excel('df_news.xlsx')
df_news_excel

,country,sector,analysis,search,query,title,url,sentiment
0,United States,Agricultural crops and livestock,Political News Article,United States Agricultural crops and livestock...,United States Agricultural crops and livestock...,Weekly Agriculture - POLITICO,https://www.politico.com/newsletters/weekly-ag...,1
1,United States,Agricultural crops and livestock,Political News Article,United States Agricultural crops and livestock...,United States Agricultural crops and livestock...,Farming | US News,https://www.usnews.com/topics/subjects/farming,-1
2,United States,Agricultural crops and livestock,Political News Article,United States Agricultural crops and livestock...,United States Agricultural crops and livestock...,American Farmers Are in Crisis. Here's Why | Time,https://time.com/5736789/small-american-farmer...,1
3,United States,Agricultural crops and livestock,Political News Article,United States Agricultural crops and livestock...,United States Agricultural crops and livestock...,Agriculture and Farming - The New York Times,https://www.nytimes.com/topic/subject/agricult...,-1
4,United States,Agricultural crops and livestock,Political News Article,United States Agricultural crops and livestock...,United States Agricultural crops and livestock...,A Glimpse into the 2022 Agricultural Economy |...,https://agamerica.com/blog/2022-agricultural-e...,1
...,...,...,...,...,...,...,...,...
13847,Singapore,Domestic and household services,Legal News Article,Singapore Domestic and household services Lega...,Singapore Domestic and household services Lega...,Latest Legal News | Today's Breaking Headlines...,https://www.reuters.com/legal/,-1
13848,Singapore,Domestic and household services,Legal News Article,Singapore Domestic and household services Lega...,Singapore Domestic and household services Lega...,Domestic Violence Law in Singapore - IRB Law,https://irblaw.com.sg/learning-centre/domestic...,-1
13849,Singapore,Domestic and household services,Legal News Article,Singapore Domestic and household services Lega...,Singapore Domestic and household services Lega...,Recycling bins to be given to each household t...,https://www.straitstimes.com/singapore/environ...,-1
13850,Singapore,Domestic and household services,Legal News Article,Singapore Domestic and household services Lega...,Singapore Domestic and household services Lega...,Singapore domestic workers' day off - BBC News,https://www.bbc.com/news/world-asia-24216611,-1


In [24]:
# Export Dataframes to Dataset on pgAdmin

# Connect to the database
conn = psycopg2.connect(
    host="localhost",
    database="Final Project",
    user="postgres",
    password=""
)

# Open a cursor to perform database operations
cur = conn.cursor()

# Define the SQL command to create the first table
create_table_query1 = '''
CREATE TABLE sentiment_data (
    sentiment INTEGER NOT NULL,
    number BIGINT NOT NULL,
    date TIMESTAMP NOT NULL,
    no_query VARCHAR(50) NOT NULL,
    username VARCHAR(50) NOT NULL,
    title VARCHAR(255) NOT NULL
)
'''

# Execute the SQL command to create the first table
cur.execute(create_table_query1)

# Define the SQL command to create the second table
create_table_query2 = '''
CREATE TABLE news_data (
    country VARCHAR(50) NOT NULL,
    sector VARCHAR(50) NOT NULL,
    analysis VARCHAR(50) NOT NULL,
    search VARCHAR(255) NOT NULL,
    query VARCHAR(255) NOT NULL,
    title VARCHAR(255) NOT NULL,
    url VARCHAR(255) NOT NULL,
    sentiment INTEGER NOT NULL
)
'''

# Execute the SQL command to create the second table
cur.execute(create_table_query2)

# Define the SQL command to insert data into the first table
insert_command1 = '''
INSERT INTO sentiment_data (sentiment, number, date, no_query, username, title) 
VALUES (%s, %s, %s, %s, %s, %s)
'''

# Define the SQL command to insert data into the second table
insert_command2 = '''
INSERT INTO news_data (country, sector, analysis, search, query, title, url, sentiment) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
'''

# Truncate the title values in the DataFrames to 255 characters
data['title'] = data['title'].str[:255]
df_news_excel['title'] = df_news_excel['title'].str[:255]

# Truncate the url values in the DataFrames to 255 characters
df_news_excel['url'] = df_news_excel['url'].str[:255]

# Truncate the sector values in the DataFrames to 50 characters
df_news_excel['sector'] = df_news_excel['sector'].str[:50]

# Iterate over the DataFrame and insert each row into the first table
for index, row in data.iterrows():
    cur.execute(insert_command1, (row['sentiment'], row['NUMBER'], row['DATE'], row['NO_QUERY'], row['USERNAME'], row['title']))

# Iterate over the DataFrame and insert each row into the second table
for index, row in df_news_excel.iterrows():
    cur.execute(insert_command2, (row['country'], row['sector'], row['analysis'], row['search'], row['query'], row['title'], row['url'], row['sentiment']))

# Commit the changes to the database
conn.commit()

# Close the cursor and database connection
cur.close()
conn.close()

In [25]:
# Import Tables from Database to Dataframe on Python

# Connect to the database
conn = psycopg2.connect(
    host="localhost",
    database="Final Project",
    user="postgres",
    password=""
)

# Create a cursor object
cur = conn.cursor()

# Retrieve all the rows from the sentiment_data table
cur.execute("SELECT * FROM sentiment_data")
data = cur.fetchall()

# Retrieve all the rows from the news_data table
cur.execute("SELECT * FROM news_data")
df_news_excel = cur.fetchall()

# Close the cursor and database connection
cur.close()
conn.close()

# Print the first 10 rows of the sentiment_data table
print(data[:10])

# Print the first 10 rows of the news_data table
print(df_news_excel[:10])

[(-1, 1467810672, datetime.datetime(2009, 4, 6, 22, 19, 49), 'NO_QUERY', 'scotthamilton', "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"), (-1, 1467810917, datetime.datetime(2009, 4, 6, 22, 19, 53), 'NO_QUERY', 'mattycus', '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'), (-1, 1467811184, datetime.datetime(2009, 4, 6, 22, 19, 57), 'NO_QUERY', 'ElleCTF', 'my whole body feels itchy and like its on fire '), (-1, 1467811193, datetime.datetime(2009, 4, 6, 22, 19, 57), 'NO_QUERY', 'Karoli', "@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. "), (-1, 1467811372, datetime.datetime(2009, 4, 6, 22, 20), 'NO_QUERY', 'joy_wolf', '@Kwesidei not the whole crew '), (-1, 1467811592, datetime.datetime(2009, 4, 6, 22, 20, 3), 'NO_QUERY', 'mybirch', 'Need a hug '), (-1, 1467811594, datetime.datetime(2009, 4, 6, 22, 20, 3), 'NO_QUERY', 

In [26]:
# Convert the lists of tuples into pandas dataframes
data = pd.DataFrame(data, columns=["sentiment", "number", "date", "no_query", "username", "title"])
df_news_excel = pd.DataFrame(df_news_excel, columns=["country", "sector", "analysis", "search", "query", "title", "url", "sentiment"])

In [27]:
# Create X and y variables to hold the title and sentiment data, respectively
X = data['title']
y = data['sentiment']

# Create a CountVectorizer object to convert the text data into a matrix of token counts
vectorizer = CountVectorizer(stop_words='english')

# Use the fit_transform() method of the CountVectorizer object to convert the text data in X into a token count matrix
X = vectorizer.fit_transform(X)

# Split the data into training and testing sets using the train_test_split() function from sklearn
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a logistic regression model object with a maximum of 1000 iterations
clf = LogisticRegression(max_iter=1000)

# Train the logistic regression model using the fit() method and the training data
clf.fit(X_train, y_train)

# Use the predict() method to make predictions on the test data
y_pred = clf.predict(X_test)

# Use the accuracy_score() function from sklearn to calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy of the model to the console
print(f"Accuracy: {accuracy:.2f}")

# Create a confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Create a new news article title to classify
title = "Agriculture’s technology future: How connectivity can yield new growth for the country"

# Use the transform() method of the CountVectorizer object to convert the new title into a token count matrix
title_vec = vectorizer.transform([title])

# Use the predict() method of the logistic regression model to predict the sentiment of the new title
sentiment = clf.predict(title_vec)[0]

# Print the predicted sentiment of the new title to the console
print(f"Sentiment: {sentiment}")

Accuracy: 0.78
Confusion Matrix:
[[121788  37708]
 [ 32530 127974]]
Sentiment: -1


In [28]:
# Use the trained model to predict the sentiment of the titles in the df DataFrame
X_test = vectorizer.transform(df_news_excel['title'])
sentiments = clf.predict(X_test)

# Add the predicted sentiment column to the df DataFrame
df_news_excel['sentiment'] = sentiments

# Print the first five rows of the resulting DataFrame to the console
df_news_excel

,country,sector,analysis,search,query,title,url,sentiment
0,United States,Agricultural crops and livestock,Political News Article,United States Agricultural crops and livestock...,United States Agricultural crops and livestock...,Weekly Agriculture - POLITICO,https://www.politico.com/newsletters/weekly-ag...,-1
1,United States,Agricultural crops and livestock,Political News Article,United States Agricultural crops and livestock...,United States Agricultural crops and livestock...,Farming | US News,https://www.usnews.com/topics/subjects/farming,-1
2,United States,Agricultural crops and livestock,Political News Article,United States Agricultural crops and livestock...,United States Agricultural crops and livestock...,American Farmers Are in Crisis. Here's Why | Time,https://time.com/5736789/small-american-farmer...,1
3,United States,Agricultural crops and livestock,Political News Article,United States Agricultural crops and livestock...,United States Agricultural crops and livestock...,Agriculture and Farming - The New York Times,https://www.nytimes.com/topic/subject/agricult...,-1
4,United States,Agricultural crops and livestock,Political News Article,United States Agricultural crops and livestock...,United States Agricultural crops and livestock...,A Glimpse into the 2022 Agricultural Economy |...,https://agamerica.com/blog/2022-agricultural-e...,1
...,...,...,...,...,...,...,...,...
13847,Singapore,Domestic and household services,Legal News Article,Singapore Domestic and household services Lega...,Singapore Domestic and household services Lega...,Latest Legal News | Today's Breaking Headlines...,https://www.reuters.com/legal/,-1
13848,Singapore,Domestic and household services,Legal News Article,Singapore Domestic and household services Lega...,Singapore Domestic and household services Lega...,Domestic Violence Law in Singapore - IRB Law,https://irblaw.com.sg/learning-centre/domestic...,-1
13849,Singapore,Domestic and household services,Legal News Article,Singapore Domestic and household services Lega...,Singapore Domestic and household services Lega...,Recycling bins to be given to each household t...,https://www.straitstimes.com/singapore/environ...,-1
13850,Singapore,Domestic and household services,Legal News Article,Singapore Domestic and household services Lega...,Singapore Domestic and household services Lega...,Singapore domestic workers' day off - BBC News,https://www.bbc.com/news/world-asia-24216611,-1


In [30]:
# Merge the news categories with the news and their sentiment
df_news = pd.merge(news, df_news_excel, left_on=['search'], right_on=['query'], how='outer')

In [35]:
# Print the the resulting DataFrame to the console
df_news.to_excel('df_news.xlsx', index=False)

In [36]:
df_news_excel.to_csv('df_news.csv', index=False)

In [37]:
# Group the DataFrame by 'Country' and 'Sector' and calculate the mean of the 'sentiment' column
table = df_news_excel.groupby(['country', 'sector']).agg({'sentiment': 'mean'})

# Reset the index and set 'Country' as the new index
table = table.reset_index().set_index('country')

In [38]:
# Print the the resulting DataFrame to the console
table.to_csv('overall_score.csv', index=True)
table

,sector,sentiment
country,,
Denmark,Activities auxiliary to financial services and...,-0.666667
Denmark,Advertising and market research,0.694915
Denmark,Agricultural crops and livestock,0.016949
Denmark,Air transport,0.433333
Denmark,"Architectural and engineering activities, and ...",0.833333
...,...,...
United States,Transport and storage,-0.233333
United States,Transport equipment,-0.233333
United States,Water transport,-0.333333


In [ ]:
#Future Analysis
# Import the necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

# Map sentiments to -1, 0, 1
sentiment_map = {0: -1, 2: 0, 4: 1}
data['sentiment'] = data['sentiment'].map(sentiment_map)

# Split data into train and test sets
X = data['title']
y = data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Balance the training dataset using SMOTE
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Train a Random Forest Classifier
rfc = RandomForestClassifier(n_jobs=-1, random_state=42)
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', 'balanced_subsample', None]
}
grid_search = GridSearchCV(rfc, param_grid, cv=5)
grid_search.fit(X_train_res, y_train_res)

# Print best hyperparameters
print(f"Best parameters: {grid_search.best_params_}")

# Make predictions on the test set
y_pred = grid_search.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
#Future Analysis
# Import the necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC

# Create X and y variables to hold the title and sentiment data, respectively
X = data['title']
y = data['sentiment']

# Convert the text data into a matrix of token counts
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(X)

# Convert the token count matrix into a matrix of TF-IDF features
tfidf = TfidfTransformer()
X = tfidf.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Support Vector Machines model
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Use the model to make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy of the model
print(f"Accuracy: {accuracy:.2f}")

# Print the confusion matrix of the model
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Create a new news article title to classify
title = "Agriculture’s technology future: How connectivity can yield new growth for the country"

# Convert the new title into a matrix of TF-IDF features using the same vectorizer and TF-IDF transformer objects as before
title_vec = tfidf.transform(vectorizer.transform([title]))

# Use the trained SVM model to predict the sentiment of the new title
sentiment = clf.predict(title_vec)[0]

# Print the predicted sentiment of the new title to the console
print(f"Sentiment: {sentiment}")